# CS 109B Advanced Topics in Data Science, Final Project, Milestone 1

## Group 9 - Steve Robbins, Chad Tsang, and Ted Heuer
**Harvard University**<br>
**Spring 2017**<br>
**Due Date: ** Wednesday, April 5th, 2017 at 11:59pm

# Milestone 1: Getting to know your data, due Wednesday, April 5, 2017

In the beginning you should get acquainted with the data sources and do some EDA. Sign up for the TMDb [API](https://www.themoviedb.org/documentation/api), and try to download the poster of your favorite movie from within your notebook. Compare the genre entries of IMDb and TMDb for this movie and see if they are the same. Think about and write down some questions that you would like to answer in the following weeks. Keep the storytelling aspect of your final report in mind and do some pen and paper sketches about the visualizations you would like to produce. Include photographs of those sketches in your notebook. 

Most of the time a data scientist spends on a project is spend on cleaning the data. We are lucky that the data we have is already pretty clean. The Python interface to the IMDb ftp files does a lot of the additional work of cleaning as well. However, you will notice that the genre list for each movie from both databases can have different lengths. This needs to be changed in order to train a model to predict the movie genre. It is up to you to think about possible ways to address this problem and to implement one of them. There is no absolute right answer here. It depends on your interests and which questions you have in mind for the project. 

Optionally, you could also scrape additional data sources, such as Wikipedia, to obtain plot summaries. That data may give you additional useful features for genera classification. 

To guide your decision process, provide at least one visualization of how often genres are mentioned together in pairs. Your visualization should clearly show if a horror romance is more likely to occur in the data than a drama romance.

The notebook to submit for this milestone needs to at least include:

- API code to access the genre and movie poster path of your favorite movie
- Genre for this movie listed by TMDb and IMDb
- A list of the 10 most popular movies of 2016 from TMDb and their genre obtained via the API
- Comments on what challenges you see for predicting movie genre based on the data you have, and how to address them 
- Code to generate the movie genre pairs and a suitable visualization of the result
- Additional visualization sketches and EDA with a focus on movie genres
- A list of questions you could answer with this and related data. Get creative here!

The EDA questions do not necessarily have to tie into the modeling part later on. Think freely about things that might be interesting, like which actors are very specific to a genre? Are action movies more prone to producing sequels than romances? However, as you keep the focus on movie genres, think also about correlations you might discover that can help building features from the metadata for prediction. Is the length of a movie title correlated with genre?


### Install Packages

In [1]:
#!pip install IMDbPY 
#(only supported in Python 2)
# Documentation for IMDb library:  
        # http://imdbpy.sourceforge.net/support.html#documentation
        # http://imdbpy.sourceforge.net/docs/README.package.txt

In [2]:
#!pip install tmdbsimple
# Documentation for TMDb library
        # https://github.com/celiao/tmdbsimple/
        # (good resource) https://developers.themoviedb.org/3/discover/movie-discover

In [3]:
from IPython.display import Image
import urllib

from imdb import IMDb
import tmdbsimple as tmdb
tmdb.API_KEY = 'c5d41f08e55fca6e9f5fc0b6d1735540'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.cm as cmx
import matplotlib.colors as colors
from pandas.tools.plotting import scatter_matrix

### IMDb Genre & Poster

***Search for Movie by Name & Print Summary***

In [5]:
imdb = IMDb()
imdb.search_movie('lock stock')

[<Movie id:0982426[http] title:_"Lock Stock and... (1969) (TV Episode)  - Season 1 | Episode 2  - Parkin's Patch" (1969)_>,
 <Movie id:4701250[http] title:_"Lock Stock and Busted/Sign of a Home Wrecker (2015) (TV Episode)  - Season 1 | Episode 155  - Justice with Judge Mablean" (2014)_>,
 <Movie id:0243714[http] title:_"Lock, Stock..." (2000)_>,
 <Movie id:0120735[http] title:_Lock, Stock and Two Smoking Barrels (1998)_>,
 <Movie id:0157472[http] title:_Clockstoppers (2002)_>,
 <Movie id:6231970[http] title:_"Rock Story" (2016)_>,
 <Movie id:0065993[http] title:_Lock, Stock and Barrel (1971) (TV)_>,
 <Movie id:3592354[http] title:_Clocks Tell the Time (2014)_>,
 <Movie id:0374557[http] title:_C Block Story (2003)_>,
 <Movie id:3327432[http] title:_Rock Story (I) (2015)_>,
 <Movie id:0050972[http] title:_Silk Stockings (1957)_>,
 <Movie id:4338022[http] title:_Back Stock (2014)_>,
 <Movie id:0021752[http] title:_The Clock Store (1931)_>,
 <Movie id:0256346[http] title:_Rockstock (2000) 

In [6]:
lock_stock_imdb_id = '0120735'
lock_stock_imdb = imdb.get_movie(lock_stock_imdb_id)
lock_stock_imdb.summary()

u"Movie\n=====\nTitle: Lock, Stock and Two Smoking Barrels (1998)\nGenres: Comedy, Crime.\nDirector: Guy Ritchie.\nWriter: Guy Ritchie.\nCast: Jason Flemyng (Tom), Dexter Fletcher (Soap), Nick Moran (Eddy), Jason Statham (Bacon), Steven Mackintosh (Winston).\nRuntime: 107, 120::(director's cut).\nCountry: UK.\nLanguage: English.\nRating: 8.2 (436423 votes).\nPlot: Four Jack-the-lads find themselves heavily - seriously heavily - in debt to an East End hard man and his enforcers after a crooked card game. Overhearing their neighbours in the next flat plotting to hold up a group of out-of-their-depth drug growers, our heros decide to stitch up the robbers in turn. In a way the confusion really starts when a pair of antique double-barrelled shotguns go missing in a completely different scam."

***Print Genres and Poster***

In [7]:
lock_stock_imdb['genres']

[u'Comedy', u'Crime']

In [8]:
lock_stock_imdb_poster_path = lock_stock_imdb['full-size cover url']
print(lock_stock_imdb_poster_path)

https://images-na.ssl-images-amazon.com/images/M/MV5BMTAyN2JmZmEtNjAyMy00NzYwLThmY2MtYWQ3OGNhNjExMmM4XkEyXkFqcGdeQXVyNDk3NzU2MTQ@.jpg


In [9]:
Image(lock_stock_imdb_poster_path)

### TMDb Exploration

***Search for Movie by Name & Print Summary***

In [10]:
def getMoviesByString(title_text):
    search = tmdb.Search()
    response = search.movie(query=title_text)
    return search.results

In [11]:
def getMoviesByTMDBId(tmdb_id):
    movie = tmdb.Movies(tmdb_id)
    return movie.info()

In [12]:
def getMoviesByIMDBId(imdb_id):
    find = tmdb.Find(id=imdb_id)
    response = find.info(external_source="imdb_id")
    return response['movie_results']

In [13]:
getMoviesByString('lock two') # search to yield more than one result...for sake of understanding output

[{u'adult': False,
  u'backdrop_path': u'/kzeR7BA0htJ7BeI6QEUX3PVp39s.jpg',
  u'genre_ids': [35, 80],
  u'id': 100,
  u'original_language': u'en',
  u'original_title': u'Lock, Stock and Two Smoking Barrels',
  u'overview': u'A card sharp and his unwillingly-enlisted friends need to make a lot of cash quick after losing a sketchy poker match. To do this they decide to pull a heist on a small-time gang who happen to be operating out of the flat next door.',
  u'popularity': 1.752122,
  u'poster_path': u'/qV7QaSf7f7yC2lc985zfyOJIAIN.jpg',
  u'release_date': u'1998-03-05',
  u'title': u'Lock, Stock and Two Smoking Barrels',
  u'video': False,
  u'vote_average': 7.4,
  u'vote_count': 1398},
 {u'adult': False,
  u'backdrop_path': None,
  u'genre_ids': [10751],
  u'id': 324645,
  u'original_language': u'en',
  u'original_title': u'Four Winds Island Part Two The Lockwood Jewels',
  u'overview': u"Adventure serial about a school girl's search for lost jewels on a small Sicilian island.",
  u'po

In [14]:
lock_stock_tmdb_id = 100
lock_stock_tmdb = getMoviesByTMDBId(lock_stock_tmdb_id)
lock_stock_tmdb

{u'adult': False,
 u'backdrop_path': u'/kzeR7BA0htJ7BeI6QEUX3PVp39s.jpg',
 u'belongs_to_collection': None,
 u'budget': 1350000,
 u'genres': [{u'id': 35, u'name': u'Comedy'}, {u'id': 80, u'name': u'Crime'}],
 u'homepage': u'http://www.universalstudiosentertainment.com/lock-stock-and-two-smoking-barrels/',
 u'id': 100,
 u'imdb_id': u'tt0120735',
 u'original_language': u'en',
 u'original_title': u'Lock, Stock and Two Smoking Barrels',
 u'overview': u'A card sharp and his unwillingly-enlisted friends need to make a lot of cash quick after losing a sketchy poker match. To do this they decide to pull a heist on a small-time gang who happen to be operating out of the flat next door.',
 u'popularity': 0.752122,
 u'poster_path': u'/qV7QaSf7f7yC2lc985zfyOJIAIN.jpg',
 u'production_companies': [{u'id': 146, u'name': u'Handmade Films Ltd.'},
  {u'id': 491, u'name': u'Summit Entertainment'},
  {u'id': 1382, u'name': u'PolyGram Filmed Entertainment'},
  {u'id': 13419, u'name': u'SKA Films'},
  {u'id'

In [15]:
getMoviesByIMDBId("tt"+lock_stock_imdb_id) # to demonstrate that we can pull TMDb info with IMDb id

[{u'adult': False,
  u'backdrop_path': u'/kzeR7BA0htJ7BeI6QEUX3PVp39s.jpg',
  u'genre_ids': [35, 80],
  u'id': 100,
  u'original_language': u'en',
  u'original_title': u'Lock, Stock and Two Smoking Barrels',
  u'overview': u'A card sharp and his unwillingly-enlisted friends need to make a lot of cash quick after losing a sketchy poker match. To do this they decide to pull a heist on a small-time gang who happen to be operating out of the flat next door.',
  u'popularity': 1.752122,
  u'poster_path': u'/qV7QaSf7f7yC2lc985zfyOJIAIN.jpg',
  u'release_date': u'1998-03-05',
  u'title': u'Lock, Stock and Two Smoking Barrels',
  u'video': False,
  u'vote_average': 7.4,
  u'vote_count': 1398}]

***Print Genres and Poster***

In [16]:
def getMoviePosterURL(movie):
    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w500"
    poster_path = movie['poster_path']
    poster_url = base_url+file_size+poster_path
    return poster_url

In [17]:
def getMoviePoster(movie):
    return getMoviePosterByPath(movie['poster_path'], movie['title'])

In [18]:
lock_stock_tmdb['genres']

[{u'id': 35, u'name': u'Comedy'}, {u'id': 80, u'name': u'Crime'}]

In [19]:
lock_stock_tmdb_poster_path = getMoviePosterURL(lock_stock_tmdb)
print(lock_stock_tmdb_poster_path)

https://image.tmdb.org/t/p/w500/qV7QaSf7f7yC2lc985zfyOJIAIN.jpg


In [20]:
Image(lock_stock_tmdb_poster_path)

### Top 10 Movies in 2016 (By TMDb Popularity / User Score)

In [21]:
discover = tmdb.Discover()

# movies sorted by user votes...can be by TMDb's "popularity score" (sort_by='popularity.desc') or...
# ...TMDb's "average user score" (sort_by='vote_average.desc')
# exclude movies with less than specified number of votes
movies_2016 = discover.movie(primary_release_year=2016, sort_by='vote_average.desc', vote_count_gte=50)

In [22]:
movies_2016

{u'page': 1,
 u'results': [{u'adult': False,
   u'backdrop_path': u'/6vkhRvsRvWpmaRVyCXaxTkIEb7j.jpg',
   u'genre_ids': [16, 18, 14, 10749],
   u'id': 372058,
   u'original_language': u'ja',
   u'original_title': u'\u541b\u306e\u540d\u306f\u3002',
   u'overview': u'High schoolers Mitsuha and Taki are complete strangers living separate lives. But one night, they suddenly switch places. Mitsuha wakes up in Taki\u2019s body, and he in hers. This bizarre occurrence continues to happen randomly, and the two must adjust their lives around each other.',
   u'popularity': 4.919579,
   u'poster_path': u'/xq1Ugd62d23K2knRUx6xxuALTZB.jpg',
   u'release_date': u'2016-08-26',
   u'title': u'Your Name.',
   u'video': False,
   u'vote_average': 8.4,
   u'vote_count': 335},
  {u'adult': False,
   u'backdrop_path': u'/w1WqcS6hT0PUWC3adG37NSUOGX5.jpg',
   u'genre_ids': [10751, 16],
   u'id': 399106,
   u'original_language': u'en',
   u'original_title': u'Piper',
   u'overview': u'A mother bird tries to 

In [23]:
num_top_movies = 10  #note: only 20 movies fit on each results page

idx = range(0, num_top_movies)
cols = ['movie_id', 'title', 'vote_average', 'tmdb_popularity', 'genres']

tmdb_df = pd.DataFrame(index=idx, columns=cols)

In [24]:
for i in range(0, num_top_movies):
    tmdb_df['movie_id'][i] = movies_2016['results'][i]['id']
    tmdb_df['title'][i] = movies_2016['results'][i]['title']
    tmdb_df['vote_average'][i] = movies_2016['results'][i]['vote_average']
    tmdb_df['tmdb_popularity'][i] = movies_2016['results'][i]['popularity']
    tmdb_df['genres'][i] = movies_2016['results'][i]['genre_ids']

In [25]:
tmdb_df

movie_id                      title vote_average tmdb_popularity  \
0   372058                 Your Name.          8.4        4.919579   
1   399106                      Piper          8.2        2.803248   
2   403450        In guerra per amore          8.1         1.16025   
3   369557                Sing Street          8.1        3.075396   
4   393729                    Divines            8        1.353409   
5   393559      My Life as a Zucchini            8        1.991033   
6   347938                   11.22.63            8         1.73654   
7   407806                       13th          7.9        1.147908   
8   290098             The Handmaiden          7.9        3.081681   
9   371645  Hunt for the Wilderpeople          7.9        3.249221   

                genres  
0  [16, 18, 14, 10749]  
1          [10751, 16]  
2      [10752, 18, 35]  
3   [10749, 18, 10402]  
4                 [18]  
5      [16, 18, 10751]  
6         [18, 36, 53]  
7                 [99]  
8      [53, 18, 10749]  
9         [18, 12, 35]

### Analyze Genre Pairs

***Table of Genres with IDs***

In [26]:
num_genres = len(tmdb.Genres().list()['genres'])

idx = range(0, num_genres)
cols = ['genre_id', 'genre']

tmdb_genre_df = pd.DataFrame(index=idx, columns=cols)

In [27]:
for i in range(0, num_genres):
    tmdb_genre_df['genre_id'][i] = tmdb.Genres().list()['genres'][i].values()[0]
    tmdb_genre_df['genre'][i] = tmdb.Genres().list()['genres'][i].values()[1]

In [28]:
tmdb_genre_df

genre_id            genre
0        28           Action
1        12        Adventure
2        16        Animation
3        35           Comedy
4        80            Crime
5        99      Documentary
6        18            Drama
7     10751           Family
8        14          Fantasy
9        36          History
10       27           Horror
11    10402            Music
12     9648          Mystery
13    10749          Romance
14      878  Science Fiction
15    10770         TV Movie
16       53         Thriller
17    10752              War
18       37          Western

***Create Table of 500 Movies with Genre Listings***

In [29]:
max_page = 25 #set to 25 since there are just over 500 entries with '2016' and 'vote_count'>=25 (20 entries per page!)
movies_per_page = 20 #always 20 entries per page

In [30]:
idx = range(0, max_page * movies_per_page)
cols = ['num_genres', 'id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6', 'id_7']

movies_genres_table = pd.DataFrame(index=idx, columns=cols)

In [31]:
for i in range(0, max_page):
    movies_page = discover.movie(page=(i+1), primary_release_year=2016, vote_count_gte=25)
    
    for j in range(0, movies_per_page):
        genre_list = movies_page['results'][j]['genre_ids']
        row_num = i * movies_per_page + j
        movies_genres_table.loc[row_num][0] = len(genre_list)

        for k in range(0, len(genre_list)):
            movies_genres_table.loc[row_num][k+1] = genre_list[k]
            

In [32]:
movies_genres_table

num_genres   id_1   id_2   id_3   id_4   id_5 id_6 id_7
0            5     16     35     18  10751  10402  NaN  NaN
1            3     12     28     14    NaN    NaN  NaN  NaN
2            4     12     16     35  10751    NaN  NaN  NaN
3            4     28     12     35  10749    NaN  NaN  NaN
4            4     28     18    878  10752    NaN  NaN  NaN
5            4     28     12     14    878    NaN  NaN  NaN
6            2     18    878    NaN    NaN    NaN  NaN  NaN
7            2     28    878    NaN    NaN    NaN  NaN  NaN
8            2     28     27    NaN    NaN    NaN  NaN  NaN
9            1     18    NaN    NaN    NaN    NaN  NaN  NaN
10           4     16     12  10751     35    NaN  NaN  NaN
11           2     36     18    NaN    NaN    NaN  NaN  NaN
12           3     18     36  10752    NaN    NaN  NaN  NaN
13           1     35    NaN    NaN    NaN    NaN  NaN  NaN
14           4     12     18  10749    878    NaN  NaN  NaN
15           4     28     12     14    878    NaN  NaN  NaN
16           4     28     12     14    878    NaN  NaN  NaN
17           3     28     12     14    NaN    NaN  NaN  NaN
18           3     18     14     12    NaN    NaN  NaN  NaN
19           1     28    NaN    NaN    NaN    NaN  NaN  NaN
20           3     18     36     53    NaN    NaN  NaN  NaN
21           5     28     80     18   9648     53  NaN  NaN
22           5     28  10752     18  10749     53  NaN  NaN
23           3     18  10749     53    NaN    NaN  NaN  NaN
24           2     18     53    NaN    NaN    NaN  NaN  NaN
25           4     28     12     14     53    NaN  NaN  NaN
26           2     28     14    NaN    NaN    NaN  NaN  NaN
27           4     28     80     14    878    NaN  NaN  NaN
28           3     80     53     18    NaN    NaN  NaN  NaN
29           2     18     35    NaN    NaN    NaN  NaN  NaN
..         ...    ...    ...    ...    ...    ...  ...  ...
470          3     53     27   9648    NaN    NaN  NaN  NaN
471          3     53     35     27    NaN    NaN  NaN  NaN
472          1     53    NaN    NaN    NaN    NaN  NaN  NaN
473          4     53     18   9648    878    NaN  NaN  NaN
474          2     27     53    NaN    NaN    NaN  NaN  NaN
475          2     36     99    NaN    NaN    NaN  NaN  NaN
476          2  10752     99    NaN    NaN    NaN  NaN  NaN
477          1     99    NaN    NaN    NaN    NaN  NaN  NaN
478          2  10402     99    NaN    NaN    NaN  NaN  NaN
479          3     28     18     53    NaN    NaN  NaN  NaN
480          1     35    NaN    NaN    NaN    NaN  NaN  NaN
481          2     16     99    NaN    NaN    NaN  NaN  NaN
482          1     18    NaN    NaN    NaN    NaN  NaN  NaN
483          1     18    NaN    NaN    NaN    NaN  NaN  NaN
484          1     35    NaN    NaN    NaN    NaN  NaN  NaN
485          3  10751     18     35    NaN    NaN  NaN  NaN
486          4     53     28     18     36    NaN  NaN  NaN
487          3    878     28     27    NaN    NaN  NaN  NaN
488          1     18    NaN    NaN    NaN    NaN  NaN  NaN
489          2     12     27    NaN    NaN    NaN  NaN  NaN
490          0    NaN    NaN    NaN    NaN    NaN  NaN  NaN
491          1     35    NaN    NaN    NaN    NaN  NaN  NaN
492          2    878     28    NaN    NaN    NaN  NaN  NaN
493          1     35    NaN    NaN    NaN    NaN  NaN  NaN
494          1     35    NaN    NaN    NaN    NaN  NaN  NaN
495          1     35    NaN    NaN    NaN    NaN  NaN  NaN
496          1     35    NaN    NaN    NaN    NaN  NaN  NaN
497          1     35    NaN    NaN    NaN    NaN  NaN  NaN
498          1     35    NaN    NaN    NaN    NaN  NaN  NaN
499          1     35    NaN    NaN    NaN    NaN  NaN  NaN

[500 rows x 8 columns]

***Create Table Showing How Genres Relate to Eachother***

In [33]:
idx = tmdb_genre_df['genre_id']
cols = tmdb_genre_df['genre_id']

genre_pairs_table = pd.DataFrame(index=idx, columns=cols)
genre_pairs_table = genre_pairs_table.fillna(0)

In [34]:
total_movies = len(movies_genres_table)

for row_num in range(0, total_movies):

    num_genres = movies_genres_table.iloc[row_num, 0]
    
    for i in range(0, num_genres):

        for j in range(0, num_genres):

            x = movies_genres_table.iloc[row_num, i+1]
            y = movies_genres_table.iloc[row_num, j+1]
            genre_pairs_table.loc[x, y] = genre_pairs_table.loc[x, y] + 1
        

In [35]:
genre_pairs_table

genre_id  28     12     16     35     80     99     18     10751  14     \
genre_id                                                                  
28          101     27      9     23     15      0     30      4     14   
12           27     60     14     21      2      0     18     15     20   
16            9     14     31     18      1      1      6     20      5   
35           23     21     18    173      5      1     52     19      9   
80           15      2      1      5     41      1     23      0      1   
99            0      0      1      1      1     23      0      0      0   
18           30     18      6     52     23      0    227      6     10   
10751         4     15     20     19      0      0      6     28      4   
14           14     20      5      9      1      0     10      4     35   
36            3      2      0      2      0      1     20      0      0   
27           12      2      0     10      4      0     11      0      4   
10402         0      0      1      9      0      3      7      2      0   
9648          4      2      0      4      6      0     13      0      2   
10749         3      6      1     25      0      0     36      1      5   
878          30     15      3      7      2      0     12      1     10   
10770         1      1      1      3      0      0      1      1      0   
53           31      8      0      9     29      0     52      0      7   
10752         6      0      0      3      0      1     11      0      0   
37            4      2      0      0      2      0      3      0      0   

genre_id  36     27     10402  9648   10749  878    10770  53     10752  37     
genre_id                                                                        
28            3     12      0      4      3     30      1     31      6      4  
12            2      2      0      2      6     15      1      8      0      2  
16            0      0      1      0      1      3      1      0      0      0  
35            2     10      9      4     25      7      3      9      3      0  
80            0      4      0      6      0      2      0     29      0      2  
99            1      0      3      0      0      0      0      0      1      0  
18           20     11      7     13     36     12      1     52     11      3  
10751         0      0      2      0      1      1      1      0      0      0  
14            0      4      0      2      5     10      0      7      0      0  
36           24      0      0      0      2      0      1      7      3      0  
27            0     70      1      8      1     11      2     37      0      0  
10402         0      1     17      1      4      1      2      1      0      0  
9648          0      8      1     22      0      1      0     13      0      0  
10749         2      1      4      0     49      2      0      5      3      0  
878           0     11      1      1      2     46      2     11      1      0  
10770         1      2      2      0      0      2      6      1      0      0  
53            7     37      1     13      5     11      1    128      5      2  
10752         3      0      0      0      3      1      0      5     17      0  
37            0      0      0      0      0      0      0      2      0      6

In [36]:
genre_pairs_table_names = genre_pairs_table.copy()

new_idx = tmdb_genre_df['genre']
new_cols = tmdb_genre_df['genre']

genre_pairs_table_names.columns = new_cols
genre_pairs_table_names = genre_pairs_table_names.set_index(new_idx)

In [37]:
genre_pairs_table_names

genre            Action  Adventure  Animation  Comedy  Crime  Documentary  \
genre                                                                       
Action              101         27          9      23     15            0   
Adventure            27         60         14      21      2            0   
Animation             9         14         31      18      1            1   
Comedy               23         21         18     173      5            1   
Crime                15          2          1       5     41            1   
Documentary           0          0          1       1      1           23   
Drama                30         18          6      52     23            0   
Family                4         15         20      19      0            0   
Fantasy              14         20          5       9      1            0   
History               3          2          0       2      0            1   
Horror               12          2          0      10      4            0   
Music                 0          0          1       9      0            3   
Mystery               4          2          0       4      6            0   
Romance               3          6          1      25      0            0   
Science Fiction      30         15          3       7      2            0   
TV Movie              1          1          1       3      0            0   
Thriller             31          8          0       9     29            0   
War                   6          0          0       3      0            1   
Western               4          2          0       0      2            0   

genre            Drama  Family  Fantasy  History  Horror  Music  Mystery  \
genre                                                                      
Action              30       4       14        3      12      0        4   
Adventure           18      15       20        2       2      0        2   
Animation            6      20        5        0       0      1        0   
Comedy              52      19        9        2      10      9        4   
Crime               23       0        1        0       4      0        6   
Documentary          0       0        0        1       0      3        0   
Drama              227       6       10       20      11      7       13   
Family               6      28        4        0       0      2        0   
Fantasy             10       4       35        0       4      0        2   
History             20       0        0       24       0      0        0   
Horror              11       0        4        0      70      1        8   
Music                7       2        0        0       1     17        1   
Mystery             13       0        2        0       8      1       22   
Romance             36       1        5        2       1      4        0   
Science Fiction     12       1       10        0      11      1        1   
TV Movie             1       1        0        1       2      2        0   
Thriller            52       0        7        7      37      1       13   
War                 11       0        0        3       0      0        0   
Western              3       0        0        0       0      0        0   

genre            Romance  Science Fiction  TV Movie  Thriller  War  Western  
genre                                                                        
Action                 3               30         1        31    6        4  
Adventure              6               15         1         8    0        2  
Animation              1                3         1         0    0        0  
Comedy                25                7         3         9    3        0  
Crime                  0                2         0        29    0        2  
Documentary            0                0         0         0    1        0  
Drama                 36               12         1        52   11        3  
Family                 1                1         1         0    0        0  
Fanta

*Observations:*

1. Of the 50 movies that listed "Romance" as a genre, 36 also listed "Drama" as a genre and just 1 listed "Horror" as a genre.
2. Of the 44 movies that listed "Science Fiction" as a genre, 29 also listed "Action" as a genre.
3. Of the 24 movites that listed "History" as a genre, 20 also listed "Drama" as a genre.
3. Movies categorized in the "Documentary" or "Horror" genres often do *not* list multiple genres, i.e. these genres are "pure".
4. For interpreting table:
    - Of the 101 "Action" movies, 27 also listed "Adventure" as a genre
    - Of the 60 "Adventure" movies, 27 also listed "Action" as a genre


***Total Number of Genre Listings for 500 Movies***

In [38]:
idx = tmdb_genre_df['genre']
cols = ['num_movies_with_genre']

genre_totals = pd.DataFrame(index=idx, columns=cols)

In [39]:
for i in range(0, len(genre_totals)):
    genre_totals.iloc[i,0] = genre_pairs_table_names.ix[tmdb_genre_df['genre'][i], tmdb_genre_df['genre'][i]]

In [40]:
genre_totals

num_movies_with_genre
genre                                
Action                            101
Adventure                          60
Animation                          31
Comedy                            173
Crime                              41
Documentary                        23
Drama                             227
Family                             28
Fantasy                            35
History                            24
Horror                             70
Music                              17
Mystery                            22
Romance                            49
Science Fiction                    46
TV Movie                            6
Thriller                          128
War                                17
Western                             6

***Check that Genre Totals Match***

In [41]:
sum(genre_totals.iloc[:, 0])

1104

In [42]:
sum(movies_genres_table.loc[:,'num_genres'])

1104

***Create Heat Map***

*Unnormalized*

In [43]:
#set the same x/y axis genre to 0 
genre_pairs_plot = genre_pairs_table_names.copy()

for idx in new_idx:
    genre_pairs_plot.set_value(idx, idx, 0)
    
genre_pairs_plot

genre            Action  Adventure  Animation  Comedy  Crime  Documentary  \
genre                                                                       
Action                0         27          9      23     15            0   
Adventure            27          0         14      21      2            0   
Animation             9         14          0      18      1            1   
Comedy               23         21         18       0      5            1   
Crime                15          2          1       5      0            1   
Documentary           0          0          1       1      1            0   
Drama                30         18          6      52     23            0   
Family                4         15         20      19      0            0   
Fantasy              14         20          5       9      1            0   
History               3          2          0       2      0            1   
Horror               12          2          0      10      4            0   
Music                 0          0          1       9      0            3   
Mystery               4          2          0       4      6            0   
Romance               3          6          1      25      0            0   
Science Fiction      30         15          3       7      2            0   
TV Movie              1          1          1       3      0            0   
Thriller             31          8          0       9     29            0   
War                   6          0          0       3      0            1   
Western               4          2          0       0      2            0   

genre            Drama  Family  Fantasy  History  Horror  Music  Mystery  \
genre                                                                      
Action              30       4       14        3      12      0        4   
Adventure           18      15       20        2       2      0        2   
Animation            6      20        5        0       0      1        0   
Comedy              52      19        9        2      10      9        4   
Crime               23       0        1        0       4      0        6   
Documentary          0       0        0        1       0      3        0   
Drama                0       6       10       20      11      7       13   
Family               6       0        4        0       0      2        0   
Fantasy             10       4        0        0       4      0        2   
History             20       0        0        0       0      0        0   
Horror              11       0        4        0       0      1        8   
Music                7       2        0        0       1      0        1   
Mystery             13       0        2        0       8      1        0   
Romance             36       1        5        2       1      4        0   
Science Fiction     12       1       10        0      11      1        1   
TV Movie             1       1        0        1       2      2        0   
Thriller            52       0        7        7      37      1       13   
War                 11       0        0        3       0      0        0   
Western              3       0        0        0       0      0        0   

genre            Romance  Science Fiction  TV Movie  Thriller  War  Western  
genre                                                                        
Action                 3               30         1        31    6        4  
Adventure              6               15         1         8    0        2  
Animation              1                3         1         0    0        0  
Comedy                25                7         3         9    3        0  
Crime                  0                2         0        29    0        2  
Documentary            0                0         0         0    1        0  
Drama                 36               12         1        52   11        3  
Family                 1                1         1         0    0        0  
Fanta

In [44]:
data = genre_pairs_plot
fig, ax = plt.subplots(figsize = (20,10))
heatmap = ax.pcolor(data, cmap=plt.cm.Blues)
ax.set_xticks(np.arange(data.shape[0])+0.5, minor = False)
ax.set_yticks(np.arange(data.shape[1])+0.5, minor = False)
ax.invert_yaxis()
ax.xaxis.tick_top()
ax.set_xticklabels(new_idx, minor=False)
ax.set_yticklabels(new_cols, minor=False)
plt.show()

/Users/ted/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


*Normalized Table and Heat Map*

In [45]:
genre_pairs_plot

genre            Action  Adventure  Animation  Comedy  Crime  Documentary  \
genre                                                                       
Action                0         27          9      23     15            0   
Adventure            27          0         14      21      2            0   
Animation             9         14          0      18      1            1   
Comedy               23         21         18       0      5            1   
Crime                15          2          1       5      0            1   
Documentary           0          0          1       1      1            0   
Drama                30         18          6      52     23            0   
Family                4         15         20      19      0            0   
Fantasy              14         20          5       9      1            0   
History               3          2          0       2      0            1   
Horror               12          2          0      10      4            0   
Music                 0          0          1       9      0            3   
Mystery               4          2          0       4      6            0   
Romance               3          6          1      25      0            0   
Science Fiction      30         15          3       7      2            0   
TV Movie              1          1          1       3      0            0   
Thriller             31          8          0       9     29            0   
War                   6          0          0       3      0            1   
Western               4          2          0       0      2            0   

genre            Drama  Family  Fantasy  History  Horror  Music  Mystery  \
genre                                                                      
Action              30       4       14        3      12      0        4   
Adventure           18      15       20        2       2      0        2   
Animation            6      20        5        0       0      1        0   
Comedy              52      19        9        2      10      9        4   
Crime               23       0        1        0       4      0        6   
Documentary          0       0        0        1       0      3        0   
Drama                0       6       10       20      11      7       13   
Family               6       0        4        0       0      2        0   
Fantasy             10       4        0        0       4      0        2   
History             20       0        0        0       0      0        0   
Horror              11       0        4        0       0      1        8   
Music                7       2        0        0       1      0        1   
Mystery             13       0        2        0       8      1        0   
Romance             36       1        5        2       1      4        0   
Science Fiction     12       1       10        0      11      1        1   
TV Movie             1       1        0        1       2      2        0   
Thriller            52       0        7        7      37      1       13   
War                 11       0        0        3       0      0        0   
Western              3       0        0        0       0      0        0   

genre            Romance  Science Fiction  TV Movie  Thriller  War  Western  
genre                                                                        
Action                 3               30         1        31    6        4  
Adventure              6               15         1         8    0        2  
Animation              1                3         1         0    0        0  
Comedy                25                7         3         9    3        0  
Crime                  0                2         0        29    0        2  
Documentary            0                0         0         0    1        0  
Drama                 36               12         1        52   11        3  
Family                 1                1         1         0    0        0  
Fanta

In [46]:
norm_genre_pairs_plot = genre_pairs_plot.copy()

In [47]:
norm_genre_pairs_plot

genre            Action  Adventure  Animation  Comedy  Crime  Documentary  \
genre                                                                       
Action                0         27          9      23     15            0   
Adventure            27          0         14      21      2            0   
Animation             9         14          0      18      1            1   
Comedy               23         21         18       0      5            1   
Crime                15          2          1       5      0            1   
Documentary           0          0          1       1      1            0   
Drama                30         18          6      52     23            0   
Family                4         15         20      19      0            0   
Fantasy              14         20          5       9      1            0   
History               3          2          0       2      0            1   
Horror               12          2          0      10      4            0   
Music                 0          0          1       9      0            3   
Mystery               4          2          0       4      6            0   
Romance               3          6          1      25      0            0   
Science Fiction      30         15          3       7      2            0   
TV Movie              1          1          1       3      0            0   
Thriller             31          8          0       9     29            0   
War                   6          0          0       3      0            1   
Western               4          2          0       0      2            0   

genre            Drama  Family  Fantasy  History  Horror  Music  Mystery  \
genre                                                                      
Action              30       4       14        3      12      0        4   
Adventure           18      15       20        2       2      0        2   
Animation            6      20        5        0       0      1        0   
Comedy              52      19        9        2      10      9        4   
Crime               23       0        1        0       4      0        6   
Documentary          0       0        0        1       0      3        0   
Drama                0       6       10       20      11      7       13   
Family               6       0        4        0       0      2        0   
Fantasy             10       4        0        0       4      0        2   
History             20       0        0        0       0      0        0   
Horror              11       0        4        0       0      1        8   
Music                7       2        0        0       1      0        1   
Mystery             13       0        2        0       8      1        0   
Romance             36       1        5        2       1      4        0   
Science Fiction     12       1       10        0      11      1        1   
TV Movie             1       1        0        1       2      2        0   
Thriller            52       0        7        7      37      1       13   
War                 11       0        0        3       0      0        0   
Western              3       0        0        0       0      0        0   

genre            Romance  Science Fiction  TV Movie  Thriller  War  Western  
genre                                                                        
Action                 3               30         1        31    6        4  
Adventure              6               15         1         8    0        2  
Animation              1                3         1         0    0        0  
Comedy                25                7         3         9    3        0  
Crime                  0                2         0        29    0        2  
Documentary            0                0         0         0    1        0  
Drama                 36               12         1        52   11        3  
Family                 1                1         1         0    0        0  
Fanta

In [48]:
for j in range(0, len(genre_totals)):
    for i in range(0, len(genre_totals)):
        norm_genre_pairs_plot.iloc[i, j] = float(genre_pairs_plot.iloc[i, j]) / int(genre_totals.iloc[j])


In [49]:
norm_genre_pairs_plot

genre              Action  Adventure  Animation    Comedy     Crime  \
genre                                                                 
Action           0.000000   0.450000   0.290323  0.132948  0.365854   
Adventure        0.267327   0.000000   0.451613  0.121387  0.048780   
Animation        0.089109   0.233333   0.000000  0.104046  0.024390   
Comedy           0.227723   0.350000   0.580645  0.000000  0.121951   
Crime            0.148515   0.033333   0.032258  0.028902  0.000000   
Documentary      0.000000   0.000000   0.032258  0.005780  0.024390   
Drama            0.297030   0.300000   0.193548  0.300578  0.560976   
Family           0.039604   0.250000   0.645161  0.109827  0.000000   
Fantasy          0.138614   0.333333   0.161290  0.052023  0.024390   
History          0.029703   0.033333   0.000000  0.011561  0.000000   
Horror           0.118812   0.033333   0.000000  0.057803  0.097561   
Music            0.000000   0.000000   0.032258  0.052023  0.000000   
Mystery          0.039604   0.033333   0.000000  0.023121  0.146341   
Romance          0.029703   0.100000   0.032258  0.144509  0.000000   
Science Fiction  0.297030   0.250000   0.096774  0.040462  0.048780   
TV Movie         0.009901   0.016667   0.032258  0.017341  0.000000   
Thriller         0.306931   0.133333   0.000000  0.052023  0.707317   
War              0.059406   0.000000   0.000000  0.017341  0.000000   
Western          0.039604   0.033333   0.000000  0.000000  0.048780   

genre            Documentary     Drama    Family   Fantasy   History  \
genre                                                                  
Action              0.000000  0.132159  0.142857  0.400000  0.125000   
Adventure           0.000000  0.079295  0.535714  0.571429  0.083333   
Animation           0.043478  0.026432  0.714286  0.142857  0.000000   
Comedy              0.043478  0.229075  0.678571  0.257143  0.083333   
Crime               0.043478  0.101322  0.000000  0.028571  0.000000   
Documentary         0.000000  0.000000  0.000000  0.000000  0.041667   
Drama               0.000000  0.000000  0.214286  0.285714  0.833333   
Family              0.000000  0.026432  0.000000  0.114286  0.000000   
Fantasy             0.000000  0.044053  0.142857  0.000000  0.000000   
History             0.043478  0.088106  0.000000  0.000000  0.000000   
Horror              0.000000  0.048458  0.000000  0.114286  0.000000   
Music               0.130435  0.030837  0.071429  0.000000  0.000000   
Mystery             0.000000  0.057269  0.000000  0.057143  0.000000   
Romance             0.000000  0.158590  0.035714  0.142857  0.083333   
Science Fiction     0.000000  0.052863  0.035714  0.285714  0.000000   
TV Movie            0.000000  0.004405  0.035714  0.000000  0.041667   
Thriller            0.000000  0.229075  0.000000  0.200000  0.291667   
War                 0.043478  0.048458  0.000000  0.000000  0.125000   
Western             0.000000  0.013216  0.000000  0.000000  0.000000   

genre              Horror     Music   Mystery   Romance  Science Fiction  \
genre                                                                      
Action           0.171429  0.000000  0.181818  0.061224         0.652174   
Adventure        0.028571  0.000000  0.090909  0.122449         0.326087   
Animation        0.000000  0.058824  0.000000  0.020408         0.065217   
Comedy           0.142857  0.529412  0.181818  0.510204         0.152174   
Crime            0.057143  0.000000  0.272727  0.000000         0.043478   
Documentary      0.000000  0.176471  0.000000  0.000000         0.000000   
Drama            0.157143  0.411765  0.590909  0.734694         0.260870   
Family           0.000000  0.117647  0.000000  0.020408         0.021739   
Fantasy          0.057143  0.000000  0.090909  0.102041         0.217391   
History          0.000000  0.000000  0.000000  0.040816         0.000000   
Horror           0.000000  0.058824  0.363636  0.020408         0.239130   
Music  

In [50]:
new_cols_chart = tmdb_genre_df['genre']
new_cols_chart[2] = 'Animat.'
new_cols_chart[5] = 'Docum.'
new_cols_chart[14] = 'Sci Fi'

In [51]:
new_cols_chart

0        Action
1     Adventure
2       Animat.
3        Comedy
4         Crime
5        Docum.
6         Drama
7        Family
8       Fantasy
9       History
10       Horror
11        Music
12      Mystery
13      Romance
14       Sci Fi
15     TV Movie
16     Thriller
17          War
18      Western
Name: genre, dtype: object

In [52]:
data = norm_genre_pairs_plot
fig, ax = plt.subplots(figsize = (15,15))
heatmap = ax.pcolor(data, cmap=plt.cm.Blues)
ax.set_xticks(np.arange(data.shape[0])+0.5, minor = False)
ax.set_yticks(np.arange(data.shape[1])+0.5, minor = False)

ax.invert_yaxis()
ax.xaxis.tick_top()
ax.set_xticklabels(new_idx, minor=False, fontsize=9)
ax.set_yticklabels(new_cols_chart, minor=False, fontsize=9)

plt.ylabel('...Likelihood of ______ also Listed as Genre', fontsize=20)
plt.text(8, -0.7, 'If ______ Listed as a Genre...',
         horizontalalignment='center',
         fontsize=20)

plt.show()

In [53]:
plt.hist(movies_genres_table['num_genres'].values, 
     25)
plt.title('Count of Movies with # of Genres')
plt.xlabel('Count of Genres')
plt.ylabel('frequency')

In [55]:
tmdb_df['vote_average'][i] = movies_2016['results'][i]['vote_average']
tmdb_df['tmdb_popularity'][i] = movies_2016['results'][i]['popularity']

In [56]:
idx = range(0, max_page * movies_per_page)
cols = ['vote_average', 'popularity', 'num_genres', 'id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6', 'id_7']

g_rate = pd.DataFrame(index=idx, columns=cols)
for i in range(0, max_page):
    movies_page = discover.movie(page=(i+1), primary_release_year=2016, vote_count_gte=25)
    
    for j in range(0, movies_per_page):
        genre_list = movies_page['results'][j]['genre_ids']
        row_num = i * movies_per_page + j
        g_rate.loc[row_num][0] = movies_page['results'][j]['vote_average']
        g_rate.loc[row_num][1] = movies_page['results'][j]['popularity']
        g_rate.loc[row_num][2] = len(genre_list)

        for k in range(0, len(genre_list)):
            g_rate.loc[row_num][k+3] = genre_list[k]

In [63]:
genre_id = tmdb_genre_df

In [64]:
idx = range(0, len(genre_id))
cols = ['genre_id', 'genre', 'vote_average', 'popularity']
g_avg = pd.DataFrame(index=idx, columns=cols)


for i in range(0, len(genre_id)):
    g_avg['genre_id'][i] = tmdb_genre_df['genre_id'][i]
    g_avg['genre'][i] = tmdb_genre_df['genre'][i]
    g_avg['vote_average'][i] = float(g_rate['vote_average'][g_rate['id_1'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_2'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_3'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_4'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_5'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_6'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['vote_average'][g_rate['id_7'] == tmdb_genre_df['genre_id'][i]].sum()) / (len(g_rate['vote_average'][g_rate['id_1'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_2'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_3'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_4'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_5'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_6'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['vote_average'][g_rate['id_7'] == tmdb_genre_df['genre_id'][i]]))
    g_avg['popularity'][i] = float(g_rate['popularity'][g_rate['id_1'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_2'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_3'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_4'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_5'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_6'] == tmdb_genre_df['genre_id'][i]].sum()+
                                 g_rate['popularity'][g_rate['id_7'] == tmdb_genre_df['genre_id'][i]].sum()) / (len(g_rate['popularity'][g_rate['id_1'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_2'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_3'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_4'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_5'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_6'] == tmdb_genre_df['genre_id'][i]])+
                                 len(g_rate['popularity'][g_rate['id_7'] == tmdb_genre_df['genre_id'][i]]))

In [65]:
g_avg

genre_id      genre vote_average popularity
0        28     Action      5.80396   4.717026
1        12  Adventure        5.995   6.489297
2        16    Animat.     6.606452    6.70631
3        35     Comedy      6.10289   3.169574
4        80      Crime     5.926829   3.245199
5        99     Docum.     7.347826   1.359037
6        18      Drama     6.411454   3.329171
7     10751     Family     6.339286   7.177147
8        14    Fantasy     6.125714   6.336011
9        36    History     6.766667   3.748712
10       27     Horror     5.417143   2.299481
11    10402      Music     6.888235   6.441964
12     9648    Mystery     5.945455   2.821167
13    10749    Romance     6.391837   3.592029
14      878     Sci Fi     5.804348   5.062711
15    10770   TV Movie     5.983333   1.635317
16       53   Thriller     5.850781   2.832751
17    10752        War     6.688235   4.512131
18       37    Western     5.516667   3.195153

In [109]:
fig, ax = plt.subplots(figsize = (12,5))
genre_numbers = np.linspace(0, 1, len(new_cols_chart)) 
c_map = plt.get_cmap('rainbow') 
#get a range of colors from color map
c_norm  = colors.Normalize(vmin=0, vmax=genre_numbers[-1])
#get a map that maps a group number to a color
number_map = cmx.ScalarMappable(norm=c_norm, cmap=c_map)

#plot points colored by their group number
for genre in xrange(len(new_cols_chart)):
    #convert a group number into a color using our map
    color = number_map.to_rgba(genre/19.0)
    #make a scatter plot of a specific group colored by its group number color
    plt.scatter(g_avg.iloc[genre]['vote_average'], 
                 g_avg.iloc[genre]['popularity'], 
                 label='' + str(g_avg.iloc[genre]['genre']),
                 color=color, 
                 alpha = 1)
plt.legend(loc='center left',bbox_to_anchor=(1, 0.5))
#plt.text(x, y, g_avg['genre'])
plt.title('Scatter of Avg. Vote vs Avg. Popularity by Genre') 
plt.xlabel('Avg. Vote')
plt.ylabel('Avg. Popularity')

### Addressing the Challenges for Predicting Movie Genres

Several challenges exist for predicting movie genres. 
- **Multiple genres per movie:** This is the most obvious problem. Above we pulled in genre information from 500 movies from 2016; a total of 1,125 genres were listed. A possible solution would be to combine genres. As shown in the heat map above, however, there are correlations between movie genres. For example, of the 24 movies listed in the "History" genre, 20 were also listed in the "Drama" genre. Combining "History" movies into the larger "Drama" category may make sense. The same logic can be applied for "Sci Fi" movies being combined into the "Action" category, and for "Crime" movies being combined into the "Thriller" category. Another possible solution is to use other sources than TMDb to determine the most frequently mentioned genre. However, as discussed below, this comes with additional issues.
- **Different genre categories by source:** IMDb categorizes movies into major genres and sub-genres, while TMDb does not use sub-genres. Furthermore, the genres are different. We can handle this by creating a set of genres that (a) is broad enough to capture every movie, and (b) is small enough (i.e. a managable number of genres) so that there is little overlap between each genre.
- **Subjectivity of genre definitions:** Genre definitions are subjective in nature and open to human interpretation.  As such, different contributors of information may classify a movie's genre differently.
- **Different movie posters:** As shown above for "Lock, Stock...", the posters given from IMDb and TMDb may be different. This may pose a problem when predicting the movie genre from posters.
- **Lots of information:** From IMDb and TMDb alone, there is a lot of information given for each movie: actors, directors, plot summaries, poster images, etc. We must determine which information we want to focus on for making genre predictions.

### Additional visualization sketches and EDA with a focus on movie genres

**Visualization Brainstorming**
![First Sketch](Milestone1-EDASketches/Sketch1.jpg)
![First Sketch](Milestone1-EDASketches/Sketch2.jpg)
![First Sketch](Milestone1-EDASketches/Sketch3.jpg)
![First Sketch](Milestone1-EDASketches/Sketch4.jpg)

**Preliminary Genre-based EDA**
![First Sketch](Milestone1-EDASketches/EDA-Dashboard.png)

### List of Questions You Could Answer with This and Related Data

With this and other data, we expect to explore answering the following questions: 
- With what accuracy can we predict movie genres given the poster image *only*.
- Can we predict genres with just quantitative data such as votes, popularity, film duration, and etc?
- Would attributes such as lead actor, studio, and title word counts be useful in the prediction of genres?
- Does the month of release impact the likelihood of genre?
- Do the text of the overview, when used as a bag of words (and dimensionality reduction) inform the genre?

### Appendix - Additional Code Snippets

In [67]:
def getMoviePosterByPath(poster_path, filename):
    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w500"
    poster_uri = base_url+file_size+poster_path
    local_filname = filename + ".jpg"
    # Download and save file locally.
    urllib.urlretrieve(poster_uri, local_filname)
    return local_filname

In [68]:
def loadGenres():
    return tmdb.Genres().list()

In [69]:
print loadGenres()

{u'genres': [{u'id': 28, u'name': u'Action'}, {u'id': 12, u'name': u'Adventure'}, {u'id': 16, u'name': u'Animation'}, {u'id': 35, u'name': u'Comedy'}, {u'id': 80, u'name': u'Crime'}, {u'id': 99, u'name': u'Documentary'}, {u'id': 18, u'name': u'Drama'}, {u'id': 10751, u'name': u'Family'}, {u'id': 14, u'name': u'Fantasy'}, {u'id': 36, u'name': u'History'}, {u'id': 27, u'name': u'Horror'}, {u'id': 10402, u'name': u'Music'}, {u'id': 9648, u'name': u'Mystery'}, {u'id': 10749, u'name': u'Romance'}, {u'id': 878, u'name': u'Science Fiction'}, {u'id': 10770, u'name': u'TV Movie'}, {u'id': 53, u'name': u'Thriller'}, {u'id': 10752, u'name': u'War'}, {u'id': 37, u'name': u'Western'}]}


In [70]:
def getMoviesByGenreId(genre_id):
    return tmdb.Genres(id=genre_id).movies()

In [71]:
print getMoviesByGenreId(28)

{u'total_results': 19275, u'total_pages': 964, u'results': [{u'poster_path': u'/45Y1G5FEgttPAwjTYic6czC9xCn.jpg', u'title': u'Logan', u'overview': u"In the near future, a weary Logan cares for an ailing Professor X in a hide out on the Mexican border. But Logan's attempts to hide from the world and his legacy are up-ended when a young mutant arrives, being pursued by dark forces.", u'release_date': u'2017-02-28', u'popularity': 112.854142, u'original_title': u'Logan', u'backdrop_path': u'/5pAGnkFYSsFJ99ZxDIYnhQbQFXs.jpg', u'vote_count': 2098, u'video': False, u'adult': False, u'vote_average': 7.6, u'genre_ids': [28, 18, 878], u'id': 263115, u'original_language': u'en'}, {u'poster_path': u'/5wBbdNb0NdGiZQJYoKHRv6VbiOr.jpg', u'title': u'Kong: Skull Island', u'overview': u'Explore the mysterious and dangerous home of the king of the apes as a team of explorers ventures deep inside the treacherous, primordial island.', u'release_date': u'2017-03-08', u'popularity': 57.579741, u'original_ti

In [72]:
movie = getMoviesByString("The Bourne Ult")
print "Genres for " + movie[0]['title'] + " include: " 
print movie[0]['genre_ids']
poster_local_filname = getMoviePosterByPath(movie[0]['poster_path'], movie[0]['title'])
Image(poster_local_filname)

Genres for The Bourne Ultimatum include: 
[28, 18, 9648, 53]
